In [1]:
import os
import numpy as np
from scipy.signal import find_peaks
from backtesting import Backtest, Strategy
from backtesting.lib import FractionalBacktest
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Data

In [2]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

DB_URL = os.getenv('DB_URL')

engine = create_engine(
    DB_URL,
    pool_pre_ping=True,   # checks connection before using
    pool_recycle=1800,    # optional: avoids stale timeouts
    connect_args={"check_same_thread": False} if "sqlite" in DB_URL else {},
)
# engine

In [8]:
import pandas as pd
from datetime import datetime
SCHEMA = 'proddb.'
tables = {
    'p5m': SCHEMA+'coin_prices_5m',
    'p1h': SCHEMA+'coin_prices_1h',
    'f5m': SCHEMA+'f_coin_signal_5m',
    'f10m': SCHEMA+'f_coin_signal_10m',
    'f15m': SCHEMA+'f_coin_signal_15m',
    'f30m': SCHEMA+'f_coin_signal_30m',
    'f1h': SCHEMA+'f_coin_signal_1h',
    'f4h': SCHEMA+'f_coin_signal_4h',
    'f1d': SCHEMA+'f_coin_signal_1d',
    'f1D': SCHEMA+'f_coin_signal_1d',
    'orders': SCHEMA+'trade_orders_sim',
    'tp_by_sess': SCHEMA+'trade_orders_tp_by_session',
    }

def get_data(from_time:int, to_time:int, symbol, tf='f1h', extr_cols:list[str]=[]):
    try:
        table = tables[tf]
    except Exception as e:
        print(e)
        return tables['p1h']
    
    if len(extr_cols) > 0:
        extr_cols_str = ', ' + ', '.join(extr_cols)
    else:
        extr_cols_str = ''

    df = pd.read_sql(f"""
            select TO_TIMESTAMP(open_time) as open_time, open as Open, close as Close,
                high  as High, low as Low, volume as Volume 
                {extr_cols_str}
            from {table} 
            where open_time >= {from_time} and open_time < {to_time}
                and symbol = '{symbol}'
            order by open_time asc
        """,
        engine, 
        index_col="open_time")
    df.rename(columns={
        'open': 'Open',
        'close': 'Close',
        'high': 'High',
        'low': 'Low',
        'volume': 'Volume',
        'rsi7': 'rsi'
    }, inplace=True)
    return df

def get_data_4_8(symbol, tf='f1h', extr_cols=[]):
    from_time = int(datetime(2025, 4, 1, 0, 0, 0).timestamp())
    to_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
    return get_data(from_time, to_time, symbol, tf, extr_cols)

def get_data_7_8(symbol, tf='f1h', extr_cols=[]):
    from_time = int(datetime(2025, 7, 24, 0, 0, 0).timestamp())
    to_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
    return get_data(from_time, to_time, symbol, tf, extr_cols)

def get_data_9_now(symbol, tf='f1h', extr_cols=[]):
    from_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
    to_time = int(datetime.now().timestamp())
    return get_data(from_time, to_time, symbol, tf, extr_cols)

# df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
# df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])

In [9]:
from_time = int(datetime(2025, 5, 1, 0, 0, 0).timestamp())
to_time = int(datetime(2025, 10, 1, 0, 0, 0).timestamp())

get_data(from_time, to_time, 'PAXGUSDT', 'p1h')

,Open,Close,High,Low,Volume
open_time,,,,,
2025-05-13 05:00:00+00:00,3263.00,3265.00,3269.00,3258.00,145.3315
2025-05-13 06:00:00+00:00,3265.00,3270.00,3272.00,3256.00,271.2965
2025-05-13 07:00:00+00:00,3271.00,3267.00,3273.00,3263.00,123.6796
2025-05-13 08:00:00+00:00,3267.00,3267.00,3269.00,3263.00,190.6910
2025-05-13 09:00:00+00:00,3267.00,3264.00,3267.00,3262.00,78.4993
...,...,...,...,...,...
2025-09-30 12:00:00+00:00,3827.95,3828.97,3830.63,3820.97,216.1367
2025-09-30 13:00:00+00:00,3828.97,3836.49,3841.03,3824.10,153.2706
2025-09-30 14:00:00+00:00,3836.48,3862.87,3862.87,3834.91,554.3586


# Backtesting lib

In [10]:

from backtesting.test import GOOG

GOOG.tail()
def save_stats(stats, backtest, out='backtest_summary.csv',tag=''):
    d = stats.to_dict()
    _ = d.pop('_equity_curve')
    trades = d.pop('_trades')
    df = pd.DataFrame([d])
    df['tag'] = tag
    df.to_csv(out, header=False, index=False, mode='a')

    folder = tag.replace('_', '/')
    os.makedirs(folder, exist_ok=True)
    trades.to_markdown(os.path.join(folder, str(stats._strategy)+'_trades.md'), index=False)
    backtest.plot(filename=os.path.join(folder, str(stats._strategy)+'_equity_curve.html'), open_browser=False)



## Triple pattern

In [11]:
def detect_triple_pattern(df, tol=0.01, min_prominence=0.01, min_distance=5):
    """Identify Triple Top or Bottom patterns in OHLC data.
    Parameters:
    - df: DataFrame with 'Close' prices
    - kind: 'top' for Triple Top or 'bottom' for Triple Bottom
    - tol: Tolerance for matching levels as fraction
    - min_prominence: Minimum prominence for swing detection
    - min_distance: Minimum candle distance between swings
    Returns:
    - List of dictionaries with indices of pattern points and necklines
    """
    df = df.copy()
    lows = df['Low']
    highs = df['High']
    swings, _ = find_peaks(-lows, prominence=min_prominence, distance=min_distance)
    # direction = 'bullish'
    bullish = []
    for i in range(len(swings) - 2):
        p1, p2, p3 = swings[i], swings[i+1], swings[i+2]
        levels = lows.values[[p1, p2, p3]]
        lv_mean = levels.mean()
        if np.ptp(levels) <= tol * lv_mean:
            neckline_val = max(highs.values[p1:p3+1])
            bullish.append({
                'p1': p1, 'p2': p2, 'p3': p3,
                'boundary_value': neckline_val,
                })
            

    swings, _ = find_peaks(highs, prominence=min_prominence, distance=min_distance)
    # direction = 'bearish'
    bearish = []
    for i in range(len(swings) - 2):
        p1, p2, p3 = swings[i], swings[i+1], swings[i+2]
        levels = highs.values[[p1, p2, p3]]
        lv_mean = levels.mean()
        if np.ptp(levels) <= tol * lv_mean:
            neckline_val = min(lows.values[p1:p3+1])
            bearish.append({
                'p1': p1, 'p2': p2, 'p3': p3,
                'boundary_value': neckline_val,
                })
    return (bullish, bearish)


In [95]:
class TriplePatternStrategy(Strategy):
    max_periods = 9
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5
    def init(self,):
        df = self.data.df
        # print(self.max_periods)
        bull, bear = detect_triple_pattern(df, tol=self.tol, min_prominence=self.min_prominence, min_distance=self.min_distance)
        # print(bull, bear)
        df['bull'] = None
        
        self.bull = []
        self.bear = []

        for p in bull:
            df['bull'].iat[p['p3']] = p['boundary_value']
            self.bull.append([p['p3'], p['boundary_value']])

        df['bear'] = None
        for p in bear:
            df['bear'].iat[p['p3']] = p['boundary_value']
            self.bear.append([p['p3'], p['boundary_value']])

        # self.bull = self.I(lambda: df['bull'], name='Bullish')
        # self.bear = self.I(lambda: df['bear'], name='Bearish')

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >=  periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if len(self.bull) > 0:
            t, boundary = self.bull[0]
            if len(self.data.index) > t+1:
                if self.data.High[-1] > boundary:
                    if not self.position.is_long:
                        # print("BULL: ", self.data.index[-1], t, boundary, self.data.Close[-1], self.data.High[-1], self.data.Low[-1])
                        self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
                        self.position.close()  # Close previous position if any
                    self.bull.pop(0)
                elif len(self.data.index) > t+14:
                    self.bull.pop(0)

        if len(self.bear) > 0:
            t, boundary = self.bear[0]
            if len(self.data.index) > t+1:
                if self.data.Low[-1] < boundary:
                    if not self.position.is_short:
                        # print("BEAR: ", self.data.index[-1], t, boundary, self.data.Close[-1], self.data.High[-1], self.data.Low[-1])
                        # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
                        self.position.close()  # Close previous position if any
                    self.bear.pop(0)
                elif len(self.data.index) > t+14:
                    self.bear.pop(0)

class TriplePatternStrategy_4p(TriplePatternStrategy):
    max_periods = 4
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5

class TriplePatternStrategy_9p(TriplePatternStrategy):
    max_periods = 9
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5

class TriplePatternStrategy_26p(TriplePatternStrategy):
    max_periods = 26
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5
 

In [96]:
# df = GOOG.copy()

# bt = FractionalBacktest(df, TriplePatternStrategy_26p, cash=10_000, commission=.002, fractional_unit=0.001)
# stats = bt.run()
# stats.to_csv("backtest_results.csv", )


In [295]:
# bt.plot()

## Butterfly Pattern

In [316]:
from pyharmonics.marketdata import BinanceCandleData  # or your own DataFrame
from pyharmonics.technicals import Technicals
from pyharmonics.search import HarmonicSearch
import pandas as pd

def detect_butterfly_pattern(df):
    _df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    _df.columns = ['open', 'high', 'low', 'close', 'volume']
    t = Technicals(_df, 'SYMBOL', 'CUSTOM')  # timeframe as needed
    hs = HarmonicSearch(t)
    hs.search()
    patterns = hs.get_patterns(family=hs.XABCD)  # includes Butterfly 
    return patterns['XABCD']

In [317]:
class ButterflyStrategy(Strategy):
    max_periods = 9
    def init(self):
        df = self.data.df
        # print(self.max_periods)
        patterns = detect_butterfly_pattern(df)
        # print(patterns)
        df['bull'] = 0
        df['bear'] = 0
        if len(patterns) > 0:            
            for p in patterns:
                if p.bullish:
                    df['bull'].at[p.x[-1]] = 1
                else:
                    df['bear'].at[p.x[-1]] = 1

        self.bull = self.I(lambda: df['bull'], name='Bullish')
        self.bear = self.I(lambda: df['bear'], name='Bearish')

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if self.bull > 0:
            if not self.position.is_long:
                self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
                self.position.close()  # Close previous position if any
        if self.bear > 0:
            if not self.position.is_short:
                # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
                self.position.close()  # Close previous position if any

class ButterflyStrategy_4p(ButterflyStrategy):
    max_periods = 4

class ButterflyStrategy_9p(ButterflyStrategy):
    max_periods = 9

class ButterflyStrategy_26p(ButterflyStrategy):
    max_periods = 26
 

In [298]:
# bt = FractionalBacktest(df, ButterflyStrategy_9p, cash=10_000, commission=.002, fractional_unit=0.01)
# stats = bt.run()
# bt.plot()


## RSI long Pattern

In [299]:
class RSILongStrategy(Strategy):
    rsi_periods = 7
    max_periods = 9
    def init(self):
        self.rsi = self.I(lambda x: x, self.data.rsi)

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if len(self.rsi) < 3:
            return
        rsi_left = self.rsi[-5:-2]
        rsi_edge = self.rsi[-2]
        rsi_right = self.rsi[-1]
        # print("rsi: ", self.rsi[-10:])

        if rsi_right >= 75:
            if rsi_edge > rsi_right and rsi_edge >= max(rsi_left):
                self.position.close()
                # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
        elif rsi_right <= 25:
            if rsi_edge < rsi_right and rsi_edge <= min(rsi_left):
                self.position.close()
                self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP

class RSI7LongStrategy_4p(RSILongStrategy):
    rsi_periods = 7
    max_periods = 4

class RSI7LongStrategy_9p(RSILongStrategy):
    rsi_periods = 7
    max_periods = 9

class RSI7LongStrategy_26p(RSILongStrategy):
    rsi_periods = 7
    max_periods = 26
# stras = RSI7LongStrategy_9p(data = df)
# stras.data

## ADX long Pattern

In [381]:
class ADXLongStrategy(Strategy):
    max_periods = 9
    def init(self):
        self.adx = self.I(lambda x: x, self.data.adx)

        predict_trend = []
        for i in range(len(self.data.index)):
            if i < 5:
                predict_trend.append(0)
                continue

            if min(self.data.adx[i-4:i+1]) == self.data.adx[i-1] or max(self.data.adx[i-4:i+1]) == self.data.adx[i-1]:  # edge point
                # low_trend = self.data.Low[i-5:i-1] < self.data.Low[i-1]
                # high_trend = self.data.High[i-5:i-1] < self.data.High[i-1]
                # close_trend = self.data.Close[i-5:i-1] < self.data.Close[i-1]
                # if (low_trend+high_trend+close_trend) >= 2:  # current trend up 
                low_trend = self.data.Low[i-5:i-1].mean() < self.data.Low[i-1]
                high_trend = self.data.High[i-5:i-1].mean() < self.data.High[i-1]
                close_trend = self.data.Close[i-5:i-1].mean() < self.data.Close[i-1]
                if (int(low_trend)+int(high_trend)+int(close_trend)) >= 2:  # current trend up 
                    predict_trend.append(-1)                
                else:  # current trend down 
                    predict_trend.append(1)
            else:
                predict_trend.append(0)

        self.predict_trend = self.I(lambda x: x, predict_trend)



    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) - 1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        # if len(self.predict_trend) >18:
            # print(self.data.Close)
            # print(self.adx)
            # print(self.predict_trend)
        if self.predict_trend == 1:
            if not self.position.is_long:
                self.buy(sl=self.data.Close[-1] * 0.95)
                # sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
        elif self.predict_trend == -1:
            self.position.close()
            # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP

class ADXLongStrategy_4p(ADXLongStrategy):
    max_periods = 4

class ADXLongStrategy_9p(ADXLongStrategy):
    max_periods = 9

class ADXLongStrategy_26p(ADXLongStrategy):
    max_periods = 26
# stras = RSI7LongStrategy_9p(data = df)
# stras.data

In [ ]:
# df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
# df = get_data_9_now('BTCUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
df = get_data_7_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])

bt = FractionalBacktest(df, ADXLongStrategy_4p, cash=10_000)
# , commission=0.002
stats = bt.run()
bt.plot()


In [ ]:
df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
# df = get_data_9_now('BTCUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])

bt = FractionalBacktest(df, ADXLongStrategy_26p, cash=10_000)
# , commission=0.002
stats = bt.run()
bt.plot()


In [363]:
b = sum([1 if i != 0 else 0 for i in a])
b

372

In [ ]:
# # get_data_4_8('BTCUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
# df = get_data_9_now('BNBUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])

# bt = FractionalBacktest(df, ADXLongStrategy_4p, cash=10_000,)
# # , commission=0.002
# stats = bt.run()
# bt.plot()

# down : 172
# up 200

c = a.copy()

In [380]:
stats

Start                     2025-07-23 17:00...
End                       2025-08-31 16:00...
Duration                     38 days 23:00:00
Exposure Time [%]                    55.76923
Equity Final [$]                  10227.51642
Equity Peak [$]                   11898.31315
Return [%]                            2.27516
Buy & Hold Return [%]                -9.81837
Return (Ann.) [%]                    29.24022
Volatility (Ann.) [%]                69.75926
CAGR [%]                             23.46297
Sharpe Ratio                          0.41916
Sortino Ratio                         0.81995
Calmar Ratio                          1.87623
Alpha [%]                              6.7471
Beta                                  0.45547
Max. Drawdown [%]                   -15.58459
Avg. Drawdown [%]                    -4.80816
Max. Drawdown Duration       16 days 15:00:00
Avg. Drawdown Duration        3 days 12:00:00
# Trades                                   87
Win Rate [%]                      

## Bullish Pin Bar

In [305]:
def SMA(values, n: int) -> np.ndarray:
    """
    Return simple moving average of `values`, at each step taking into account `n` previous values.
    This returns a numpy array of the same length as values.
    """
    # Make it a pandas Series:
    s = pd.Series(values)
    sma = s.rolling(n, min_periods=1).mean()
    return sma.values  # array with same length

class BullishPinBarStrategy(Strategy):
    max_periods = 9
    wick_body_ratio = 2.0       # wick at least this × body
    min_wick_pct = 0.75          # wick at least this % of candle height
    min_range_atr = 0.8         # candle total range ≥ this × ATR
    trend_sma_period = 10       # for trend filter

    def init(self):
        # self.sma_trend = pd.Series(self.data.Close).rolling(window=self.trend_sma_period, min_periods=1).mean()
        # self.sma_trend = self.I(pd.Series.rolling, pd.Series(self.data.Close), self.trend_sma_period).mean()
        self.sma_trend = self.I(SMA, self.data.Close, self.trend_sma_period)

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) - 1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)

        body = abs(self.data.Close[-1] - self.data.Open[-1])
        upper_wick = self.data.High[-1] - max(self.data.Close[-1], self.data.Open[-1])
        lower_wick = min(self.data.Close[-1], self.data.Open[-1]) - self.data.Low[-1]

        # trend filter: define uptrend / downtrend
        # For example, price above SMA → uptrend, below → downtrend
        # print(self.data.Close, self.sma_trend)
        uptrend = self.data.Close[-1] > self.sma_trend[-1]
        downtrend = self.data.Close[-1] < self.sma_trend[-1]
        # if len(self.data.index) > 65 and len(self.data.index) < 80:
            # print(len(self.data.index), self.data.Close[-1], " - ",  self.sma_trend[-1], self.data.atr14[-1], self.data.tr[-1], body, lower_wick, upper_wick)
            # print(len(self.data.index), (lower_wick >= self.wick_body_ratio * body), uptrend, downtrend, (self.data.Close[-1] > self.data.Open[-1]))
        # require enough ATR range
        if self.data.tr[-1] < (self.min_range_atr * self.data.atr14[-1]):
            # if len(self.data.index) > 65 and len(self.data.index) < 80:
            #     print("skip, candle too small")
            return  # skip, candle too small

        # require small body relative to range
        if body > ( (1 - self.min_wick_pct) * self.data.tr[-1] ):
            # if len(self.data.index) > 65 and len(self.data.index) < 80:
            #     print("skip, body too large")
            return

        # bullish pin bar
        if (lower_wick >= self.wick_body_ratio * body) and lower_wick > upper_wick and downtrend:  #  and (self.data.Close[-1] > self.data.Open[-1]):
            # self.position.close()
            self.sell(tp=self.data.Close[-1] * 0.95)
            # sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
            # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
            # if not self.position.is_long:
            #     self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
            # self.buy(sl = sl, tp = tp)
        # elif upper_wick >= self.wick_body_ratio * body and lower_wick < upper_wick and uptrend:   #  and self.data.Open[-1] > self.data.Close[-1]
            # self.position.close()
            # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
            # if not self.position.is_long:
            # self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP

class BullishPinBarStrategy_4p(BullishPinBarStrategy):
    max_periods = 4

class BullishPinBarStrategy_9p(BullishPinBarStrategy):
    max_periods = 9

class BullishPinBarStrategy_26p(BullishPinBarStrategy):
    max_periods = 26


In [306]:
# df = get_data_4_8('ETHUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])
# bt = FractionalBacktest(df, BullishPinBarStrategy_9p, cash=10_000, commission=0.002)
# stats = bt.run()
# bt.plot()


## Gold - Candle pattern

In [150]:
class GoldCandlePattern(Strategy):
    def init(self):
        self.max_periods = 1
        # self.sma_trend = pd.Series(self.data.Close).rolling(window=self.trend_sma_period, min_periods=1).mean()
        # self.sma_trend = self.I(pd.Series.rolling, pd.Series(self.data.Close), self.trend_sma_period).mean()
        # self.sma_trend = self.I(SMA, self.data.Close, self.trend_sma_period)
    def detect_pin_bar(self, o, h, l, c):
        body = abs(c - o)
        tr = abs(h - l)
        if body < tr*0.2:
            return True
        return False

    def time_condition(self, hour_from, hour_to):
        t = self.data.index[-1]
        if t.hour >= hour_from and t.hour <= hour_to:
            return True
        return False
    
    def candle_pattern_all(self):
        o1 = self.data.Open[-2]
        h1 = self.data.High[-2]
        l1 = self.data.Low[-2]
        c1 = self.data.Close[-2]
        o2 = self.data.Open[-1]
        h2 = self.data.High[-1]
        l2 = self.data.Low[-1]
        c2 = self.data.Close[-1]
        if (c1 > o1  # c1 up
            and c2 > o2  # c2 up
            and h2 > h1
            and (c2 > h1 or l2 > l1)
            ):
            entry = [{'price': c2, 'qty': 1}]
            if o2 - l2 > h2 - o2:
                entry[0]['qty'] = 0.5
                entry.append({'price':(c2+o2)/2, 'qty': 0.5})
            return (1, entry)
        elif (c2 < o2  # c2 dowm
            and (h1 < h2 or c1 < l2)
            and h1 < h2
            ):
            entry = [{'price': c2, 'qty': 1}]
            if h2 - c2 > c2 - l2:
                entry[0]['qty'] = 0.5
                entry.append({'price':(c2+o2)/2, 'qty': 0.5})
            return (-1, entry)
        else:
            return 0

    def candle_pattern_1(self):
        o1 = self.data.Open[-2]
        h1 = self.data.High[-2]
        l1 = self.data.Low[-2]
        c1 = self.data.Close[-2]
        o2 = self.data.Open[-1]
        h2 = self.data.High[-1]
        l2 = self.data.Low[-1]
        c2 = self.data.Close[-1]

        if (c1 > o1  # c1 up
            and c2 > o2  # c2 up
            and h2 > h1
            and (c2 > h1 and l2 > l1)
            ):
            entry = [{'price': c2, 'qty': 1}]
            if o2 - l2 > h2 - o2:
                entry[0]['qty'] = 0.5
                entry.append({'price':(c2+o2)/2, 'qty': 0.5})
            return (1, entry)
        elif (c1 < o1  # c2 dowm
            and c2 < o2  # c2 dowm
            and (h2 < h1 and c2 < l1)
            and h2 < h1
            ):
            entry = [{'price': c2, 'qty': 1}]
            if h2 - c2 > c2 - l2:
                entry[0]['qty'] = 0.5
                entry.append({'price':(c2+o2)/2, 'qty': 0.5})
            return (-1, entry)
        else:
            return (0, None)

    def CloseRevertTrend(self):
        i = max(len(self.data.Close) - 4, 0)
        trend_3 = self.data.Close[-2] - self.data.Open[i] / (len(self.data.Close) - i - 1)
        t1 = self.data.Close[-1] - self.data.Open[-1]
        t2 = self.data.Close[-2] - self.data.Open[-2]
        # print(self.data.High[-3:] - self.data.Low[-3:], atr_3)
        if self.position.is_long:
            change_rate = (self.data.Close[-1] - self.data.Open[-1]) / trend_3
            # down two candel or down > 10% atr
            if (t1<0 and t2<0) or change_rate < -0.5:
                self.position.close()
        elif self.position.is_short:
            change_rate = (self.data.Close[-1] - self.data.Open[-1]) / trend_3
            # down two candel or up > 10% atr
            if (t1>0 and t2>0) or change_rate < -0.5:
                self.position.close()
    
    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) - 1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)

    def CancelLimitOrder(self):
        for o in self.orders:
            o.cancel()

    def next(self):
        self.CloseOldPosition(self.max_periods)
        # self.CloseRevertTrend()
        if not self.time_condition(17, 18):  # 6 - 10h vn 13, 17
            return None

        l = len(self.data.index)
        for i in range(max(l-2, 0), l):
            if self.detect_pin_bar(self.data.Open[i], self.data.High[i], self.data.Low[i], self.data.Close[i]):
                return None

        signal, entrys = self.candle_pattern_1()
        # print(self.equity, signal, entrys)
        if signal == 1 and not self.position.is_long:
            tag = 'cancel_short' if self.position.is_short else 'new_long'
            self.position.close()
            self.CancelLimitOrder()
            if len(self.data.Close) >= 1130 and len(self.data.Close) <= 1140:
                print(self.data.index[-1], self.position.is_short, self.position)
            sl = self.data.Low[-1]
            # self.buy(size=self.equity * entrys[0]['qty'] / entrys[0]['price'], sl=sl)
            self.buy(size=entrys[0]['qty'], sl=sl, tag=tag)
            for entry in entrys[1:]:
                # self.buy(size=self.equity * entrys[0]['qty'] / entry['price'], limit=entry['price'], sl=sl)
                self.buy(size=entrys[0]['qty'], limit=entry['price'], sl=sl, tag="limit_"+tag)
        elif signal == -1 and not self.position.is_short:
            tag = 'cancel_long' if self.position.is_long else 'new_short'
            self.position.close()
            self.CancelLimitOrder()
            if len(self.data.Close) >= 1130 and len(self.data.Close) <= 1140:
                print(self.data.index[-1], self.position.is_short, self.position)
            sl = self.data.High[-1]
            # self.sell(size=self.equity * entrys[0]['qty'] / entrys[0]['price'], sl=sl)
            self.sell(size=entrys[0]['qty'], sl=sl, tag=tag)
            for entry in entrys[1:]:                
                # self.sell(size=self.equity * entry['qty'] / entry['price'], limit=entry['price'], sl=sl)
                self.sell(size=entry['qty'], limit=entry['price'], sl=sl, tag="limit_"+tag)
        else:
            return None

# class BullishPinBarStrategy_4p(BullishPinBarStrategy):
#     max_periods = 4

# class BullishPinBarStrategy_9p(BullishPinBarStrategy):
#     max_periods = 9

# class BullishPinBarStrategy_26p(BullishPinBarStrategy):
#     max_periods = 26
# todo: new strat

In [151]:
from_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
to_time = int(datetime(2025, 10, 1, 0, 0, 0).timestamp())

df = get_data(from_time, to_time, 'PAXGUSDT', 'p5m')
bt = FractionalBacktest(df, GoldCandlePattern, cash=10_000)  #  , commission=0.002
stats = bt.run()
bt.plot()


FractionalBacktest.run:   0%|          | 0/7105 [00:00<?, ?bar/s]

GridPlot(id='p13365', ...)

In [152]:
stats
# ._trades[(stats._trades['EntryTime']>="2025-06-29 20") & (stats._trades['EntryTime']<="2025-06-30 10")]

Start                     2025-09-02 02:50...
End                       2025-09-30 16:55...
Duration                     28 days 14:05:00
Exposure Time [%]                     3.96848
Equity Final [$]                  10027.26798
Equity Peak [$]                   10037.03328
Return [%]                            0.27268
Buy & Hold Return [%]                 9.56771
Return (Ann.) [%]                     4.04803
Volatility (Ann.) [%]                 1.01666
CAGR [%]                              3.53802
Sharpe Ratio                           3.9817
Sortino Ratio                        11.28377
Calmar Ratio                         26.09122
Alpha [%]                             0.37075
Beta                                 -0.01025
Max. Drawdown [%]                    -0.15515
Avg. Drawdown [%]                    -0.07169
Max. Drawdown Duration       13 days 00:40:00
Avg. Drawdown Duration        3 days 11:41:00
# Trades                                  150
Win Rate [%]                      

# All in one

In [309]:
def backtest_function(symbol, timeframe, strategies, cash=10_000, commission=0.002):
    symbol = symbol.upper()
    timeframe = timeframe.lower()
    tag = f"{symbol}_{timeframe}"
    df = get_data_4_8(symbol, timeframe, ['rsi7', 'adx', 'tr', 'atr14'])
    try:
        print("df.shape: ", df.shape)
    except Exception as e:
        print("df: ", df)
        raise e
    fractional_unit = pow(10, 2-int(np.log10(df['High'].max())))
    # _stats, _plot = [], []
    for strategy in strategies:
        max_periods = strategy.max_periods
        print(f"Running backtest for {strategy.__name__} with max_periods={max_periods}")
        bt = FractionalBacktest(df, strategy, cash=cash, commission=commission, fractional_unit=fractional_unit)
        stats = bt.run()
        save_stats(stats, bt, "backtest_summary.csv", tag)

In [ ]:
a = backtest_function(
    "HBARUSDT", 
    "f1h", 
    [
        BullishPinBarStrategy_4p, BullishPinBarStrategy_9p, BullishPinBarStrategy_26p,
        ADXLongStrategy_4p, ADXLongStrategy_9p, ADXLongStrategy_26p,
        RSI7LongStrategy_4p, RSI7LongStrategy_9p, RSI7LongStrategy_26p,
        TriplePatternStrategy_4p, TriplePatternStrategy_9p, TriplePatternStrategy_26p,
        ButterflyStrategy_4p, ButterflyStrategy_9p, ButterflyStrategy_26p
    ],
    cash=10_000,
    commission=0
    )

In [ ]:
symbols = ['ETHUSDT', 'HBARUSDT']  # , 'ETHUSDT'
tfs = ["p1h"]
for symbol in symbols:
    for tf in tfs:
        a = backtest_function(
            symbol, 
            tf, 
            [TriplePatternStrategy_9p, TriplePatternStrategy_26p, ButterflyStrategy_9p, ButterflyStrategy_26p]
            )

In [ ]:
symbols = ['ETHUSDT', 'HBARUSDT']  # , 'ETHUSDT'
tfs = ["f1h"]
for symbol in symbols:
    for tf in tfs:
        a = backtest_function(symbol, tf, [ADXLongStrategy_9p, ADXLongStrategy_26p, RSI7LongStrategy_9p, RSI7LongStrategy_26p])

In [261]:
# symbols = ['BTCUSDT', 'BNBUSDT', 'ETHUSDT', 'HBARUSDT']  # , 'ETHUSDT'
# tfs = ["f1h"]
# for symbol in symbols:
#     for tf in tfs:
#         a = backtest_function(symbol, tf, [BullishPinBarStrategy_9p, BullishPinBarStrategy_26p])



,Open,Close,High,Low,Volume
open_time,,,,,
2025-05-13 00:00:00+00:00,0.20554,0.21612,0.21857,0.20419,183175906.0
2025-08-28 00:00:00+00:00,0.23762,0.23922,0.24201,0.23540,134719322.0
2025-08-29 00:00:00+00:00,0.23922,0.22796,0.24037,0.22349,249640545.0
2025-08-30 00:00:00+00:00,0.22797,0.22574,0.22951,0.22028,107200716.0
2025-08-31 00:00:00+00:00,0.22575,0.21896,0.22871,0.21784,79075844.0


In [253]:
from datetime import datetime
ts = 300
int(datetime.now().timestamp()), (int(datetime.now().timestamp()) // ts) * ts


(1758705407, 1758705300)

In [63]:
int(datetime(2025, 9, 1, 0, 0, 0).timestamp())

1756659600

In [ ]:
trasd